In [1]:
import torch
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from transformers import BertTokenizer, AutoModel

from Preprocessing import preprocess_text

# os.chdir('C:/Users/LENOVO/GitHub/Jurnal-Clustering')

c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BertClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-cased')
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state'][:, 0, :]
        x = self.classifier(x)
        return x

In [3]:
from torch.utils.data import Dataset, DataLoader

# Buat dataloader
class ArticleDataset(Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx]
        }

    def __len__(self):
        return len(self.input_ids)


In [4]:
def tokenize_data(texts, tokenizer, max_length=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            pad_to_max_length=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [2]:
df = pd.read_csv('data/extracted_publication_journal_s1.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

df


,journal,title,abstract
0,25024760,Localization schemes in Underwater Sensor Netw...,<jats:p>&lt;p&gt;Underwater Wireless Sensor Ne...
1,25024760,A Review on Voltage Balancing Solutions in Mul...,<jats:p>&lt;p&gt;Multilevel inverters are used...
2,25024760,Collision Detection and Trajectory Planning fo...,<jats:p>This paper proposes an algorithm for C...
3,25024760,A New Method for Optimal Coordination of Overc...,<jats:p>&lt;p&gt;The most of the new protectiv...
4,25024760,Fairness Evaluation and Comparison of Current ...,<jats:p>Transmission Control Protocol (TCP) is...
...,...,...,...
79988,25409581,Hesperitin Synergistically Promotes the Senesc...,"<jats:p>Pentagamavunone-1 (PGV-1), a curcumin ..."
79989,25409581,First Report on Wild Occurrences of Phoenix Mu...,<jats:p>The genus Pleurotus is known as a comm...
79990,25409581,Nannoplankton Biostratigraphy from Banggai-Sul...,<jats:p>The nannoplankton research was conduct...
79991,25409581,Spatial Modelling Habitat Suitability of Javan...,<jats:p>Javan Langur (T. auratus) is well-know...


In [12]:
df.journal.nunique()

123

In [13]:
df.journal.unique()

array(['25024760', '23387238', '2442-8620', '25278045', '27224708',
       '24610399', '26141566', '24423084', '27155072', '24769304',
       '26148013', '27765938', '2406825x', '20888708', '22528938',
       '25488465', '25023357', '24070610', '24610275', '23029277',
       '24610771', '25499904', '22524940', '24606952', '23029285',
       '24606278', '25277456', '25494333', '20854722', '24609196',
       '2338557X', '24432555', '25025791', '24606618', '26156636',
       '24078646', '23548509', '25992147', '24607010', '20888694',
       '24429899', '23556544', '25493167', '25481592', '23556145',
       '23385499', '19782993', '25989685', '24609285', '23549203',
       '24778036', '25483161', '27222594', '25483382', '25027883',
       '24774073', '27156079', '25495747', '26153386', '24067598',
       '24077542', '25992570', '24068799', '24068195', '25029266',
       '23561424', '23385510', '25279238', '23389486', '24429325',
       '2615790X', '26219468', '20864094', '25804391', '24601

In [6]:
cwd = os.getcwd()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertClassifier(12)
model.load_state_dict(torch.load('model/finetuned_model_sinta_translated.pt'))

# Membuat dataloader
batch_size = 32


In [11]:
for jour in df.journal.unique():

    journal_type = 'sinta_new_data_s1'

    # Assuming 'jurnal_id' is a variable containing the directory name
    file_path = os.path.join('src', journal_type, jour)

    if not os.path.exists(file_path):
        os.mkdir(file_path)

        data = df[df['journal'] == jour]
        data['data_cleaned'] = data['title'] + data['abstract'] 
        data['data_cleaned'].apply(lambda x : preprocess_text(x))
        X = list(data['data_cleaned'])

        input_ids, attention_masks = tokenize_data(X, tokenizer)
        dataset = ArticleDataset(input_ids, attention_masks)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        device = 'cpu'
        if torch.cuda.is_available() :
            device = 'cuda'

        model.to(device)

        # Set model ke mode evaluasi (non-training)
        model.eval()

        # Embedding
        embeddings = []

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                embeddings.append(outputs.cpu().numpy())

        embeddings = np.concatenate(embeddings, axis=0)

        # Mengubah array embeddings menjadi matriks dua dimensi
        X = embeddings.reshape(embeddings.shape[0], -1)

        pca = PCA(n_components=2, random_state=0)
        X = pca.fit_transform(X)

        # Perform KMeans clustering
        num_clusters = 1
        kmeans = KMeans(n_clusters=num_clusters, random_state=0, max_iter=1000)
        kmeans.fit(X)

        # Assign each journal to its cluster
        cluster_labels = kmeans.labels_

        # Mendapatkan koordinat pusat cluster
        centroid = kmeans.cluster_centers_

        # Menghitung jarak antara setiap titik data dengan centroid
        jarak_ke_centroid = np.sqrt(np.sum((X - centroid)**2, axis=1))

        # Menentukan batas jarak yang dianggap sebagai "outscoop"
        outscoop_threshold = np.mean(jarak_ke_centroid) + 2 * np.std(jarak_ke_centroid)

        # Memisahkan data yang masih masuk dalam "scoop" dan "outscoop"
        scoop_data = X[jarak_ke_centroid <= outscoop_threshold]
        outscoop_data = X[jarak_ke_centroid > outscoop_threshold]

        scoop_labels = np.ones(len(X))
        scoop_labels[jarak_ke_centroid > outscoop_threshold] = -1

        filename_kmeans = f"{file_path}/{jour}_kmeans.pkl"
        joblib.dump(kmeans, filename_kmeans)

        np.save(f"{file_path}/{jour}_threshold.npy", outscoop_threshold)
        np.save(f"{file_path}/{jour}_pca_data.npy", X)
        np.save(f"{file_path}/{jour}_bert_data.npy", embeddings.reshape(embeddings.shape[0], -1))

        df_res = pd.DataFrame({'Data': data['data_cleaned'],
                   'Label': scoop_labels})

        inScoop_df = df_res[df_res['Label'] == 1]
        outScoop_df = df_res[df_res['Label'] == -1]

        df_res.to_csv(f'{file_path}/{jour}_data_jurnal.csv')
        inScoop_df.to_csv(f'{file_path}/{jour}_inscoop_data_jurnal.csv')
        outScoop_df.to_csv(f'{file_path}/{jour}_outscoop_data_jurnal.csv')

        print("Data sebaran PCA {} telah disimpan.".format(jour))
    else:
        pass

    


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11024\3243461422.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 2442-8620 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11024\3243461422.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 25278045 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11024\3243461422.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

Data sebaran PCA 24600245 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11024\3243461422.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 26228645 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11024\3243461422.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 25409581 telah disimpan.


In [ ]:
# data = df[df['journal'] == '25024760']
# data['data_cleaned'] = data['title'] + data['abstract'] 
# data['data_cleaned'].apply(lambda x : preprocess_text(x))
# X = list(data['data_cleaned'])

# input_ids, attention_masks = tokenize_data(X, tokenizer)
# dataset = ArticleDataset(input_ids, attention_masks)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1868\512846536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# device = 'cpu'
# if torch.cuda.is_available() :
#     device = 'cuda'

# model.to(device)

# # Set model ke mode evaluasi (non-training)
# model.eval()

# # Embedding
# embeddings = []

# with torch.no_grad():
#     for batch in dataloader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask)
#         embeddings.append(outputs.cpu().numpy())

# embeddings = np.concatenate(embeddings, axis=0)

In [8]:




# # Mengubah array embeddings menjadi matriks dua dimensi
# X = embeddings.reshape(embeddings.shape[0], -1)

# pca = PCA(n_components=2, random_state=0)
# X = pca.fit_transform(X)

# # Perform KMeans clustering
# num_clusters = 1
# kmeans = KMeans(n_clusters=num_clusters, random_state=0, max_iter=1000)
# kmeans.fit(X)

# # Assign each journal to its cluster
# cluster_labels = kmeans.labels_

# # Mendapatkan koordinat pusat cluster
# centroid = kmeans.cluster_centers_

# # Menghitung jarak antara setiap titik data dengan centroid
# jarak_ke_centroid = np.sqrt(np.sum((X - centroid)**2, axis=1))

# # Menentukan batas jarak yang dianggap sebagai "outscoop"
# outscoop_threshold = np.mean(jarak_ke_centroid) + 2 * np.std(jarak_ke_centroid)

# # Memisahkan data yang masih masuk dalam "scoop" dan "outscoop"
# scoop_data = X[jarak_ke_centroid <= outscoop_threshold]
# outscoop_data = X[jarak_ke_centroid > outscoop_threshold]

# scoop_labels = np.ones(len(X))
# scoop_labels[jarak_ke_centroid > outscoop_threshold] = -1

# filename_kmeans = f"{file_path}/{jour}_kmeans.pkl"
# joblib.dump(kmeans, filename_kmeans)

# np.save(f"{file_path}/{jour}_threshold.npy", outscoop_threshold)
# np.save(f"{file_path}/{jour}_pca_data.npy", X)
# np.save(f"{file_path}/{jour}_bert_data.npy", embeddings.reshape(embeddings.shape[0], -1))

# df_res = pd.DataFrame({'Data': data['data_cleaned'],
#             'Label': scoop_labels})

# inScoop_df = df_res[df_res['Label'] == 1]
# outScoop_df = df_res[df_res['Label'] == -1]

# df_res.to_csv(f'{file_path}/{jour}_data_jurnal.csv')
# inScoop_df.to_csv(f'{file_path}/{jour}_inscoop_data_jurnal.csv')
# outScoop_df.to_csv(f'{file_path}/{jour}_outscoop_data_jurnal.csv')
